第一回 Pytorch クイックスタート
pytorchを用いた画像認識を体験し、深層学習の流れを学ぶ

1、ライブラリのインポート：
pythonでは以下のようなコードでライブラリをインポートして用いることができます

In [8]:
import ライブラリ名
#このコードは実行できません

ModuleNotFoundError: No module named 'ライブラリ名'

このやり方に沿ってpytorchをimportしてみよう

In [2]:
import torch

ModuleNotFoundError: No module named 'torch'

ライブラリを読み込んでも、その中でクラスや名前空間で機能が分けられている場合が多いです。
pytorchでは「torch」の中の「nn」、「DataLoader」などが頻繫に用いられているが呼び出すには
次のようにコードを記述する必要があります

In [ ]:
torch.nn
torch.utils.data.DataLoader
#このように「.」で区切って定義していく
#これは実行してもエラーは出ませんが何も起きない

しかし、いちいち「.」を使って定義するのは面倒ですよね?
よってライブラリの中でも、頻繁に使う機能に関しては工夫して
importする必要があります。以下がその例です

In [2]:
from torch import nn
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision.transforms import ToTensor

こうすると「.」を挟まず呼び出すことが可能です。
途中「torchvision」というライブラリが出てきていますが、
これはpytorchで画像を扱うときに物凄く有用なライブラリです。

In [3]:
nn
DataLoader
#このように「.」なしで呼び出してもエラーが出ない

torch.utils.data.dataloader.DataLoader

2、データセットのロード：
それではimportした機能(モジュール)を用いて、
学習用のデータと、検証用のデータを読み込んでみましょう

In [4]:
# データセットをダウンロードとデータセットの読込
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    #画像のデータをテンソル型とよぶものにする
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



ここでは「dataset」と呼ばれる機能(モジュール)の中の「FashionMNIST」クラスを読み込んでいます。
「FashionMNIST」クラスは28×28のグレースケール画像が6万枚入った画像データセットです。
クラスのインスタンスを作るだけで、勝手に画像をダウンロードし、使える状態にしてくれます。

ここで、「データセット」という単語が出てきました。
データセットとは深層学習用のデータと、その画像と紐づけられたラベル
(画像の内容を示す番号のようなもの)の集合のことです。
pytorchでは予め用意されているデータセットがいくつかあり、直ぐに使える状態になっています。

また、pytorchの場合だと、画像データを配列に直してそのまま利用することはできないので、代わりにtensor(テンソル)型と呼ばれるものに変更します。
これは「pytorch専用の配列みたいなもの」と思ってもらって大丈夫です。

3、データローダーの制作：
pytorchではデータセットの内容を読み込み、
学習時・検証時に利用するためのデータローダーというものが必要になります

In [5]:
batch_size = 64

#データローダーの制作
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

#X=画像 y=画像Xのラベル(画像の内容を示す番号のこと)
for X, y in test_dataloader:
    #N=画像の枚数 C=画像のチャンネル数 H=画像の高さ W=画像の幅
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    #y.shape=ラベルの数 y.dtype=ラベルデータのデータ型(intとかcharとか)
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


pytorchでは「DataLoader」クラスを用いてデータローダーを制作します。
Dataloaderクラスは主にデータセットとバッチサイズというものを引数に持ちます。
バッチサイズとは、データローダーが一回のロードで読み込む画像の枚数のことです。
今回の場合、データローダーのバッチサイズが64なので、6万枚の画像から64枚の画像
を読み込むようになっています。
なぜ、6万枚一気に読み込まないのかは「ミニバッチ法」と呼ばれる学習方法に由来するためです。（詳しくはwebか渡辺に）

4、ニューラルネットワーク(NN)の構築：
いよいよNNの構築に入ります。NN(ニューラルネットワーク)とは，複数のノード（点）により構成される層と，それらをつなぐエッジ（線）によって構成されるデータ構造のことで，深層学習させたAIの本体とも言ってよいでしょう．下の画像に，NNの例を示しておきます．

<img src="https://www.eaglys.co.jp/media/4dy19LFuwYpzEjRaJl8yealuEc4sAmvbM8gBPTML.jpeg">

層には，データを入力するための入力層，隠れ層（中間層），推論結果を出力する出力層の3つの層があります．各ノードには，エッジを伝ってデータが入力され，そのデータを「活性化関数」と呼ばれる関数で処理し，関数の出力結果を次の層に伝搬させます．また，エッジには重みという値が存在し，データはエッジを通るたびに，重みの分だけ値が倍増します

また，重み以外にもバイアスと呼ばれる値があります．重みが掛け算によってエッジを通るデータを変化させるのに対して，バイアスは，足し算によってデータを変化させます．

NNの出力は，この重みやバイアスの値によって変化するため，これらの値を最適化することによって出力をコントロールする事ができます．この工程を学習といいます．

pytorchではNNのテンプレートである「nn.Module」というクラスを継承することで、NNを構築することができます。

In [1]:
class NeuralNetwork(nn.Module):#nn.Moduleを継承する
    def __init__(self):
        super().__init__()#継承先のフィールドを初期化
        self.flatten = nn.Flatten()#画像は2次元テンソルなので、28*28の要素を持つ一次元のテンソルに変換
        self.linear_relu_stack = nn.Sequential(
            #入力は28*28(画像のピクセル数)で出力は10
            nn.Linear(28*28, 512), #層（線形層）
            nn.ReLU(), #活性化関数
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

#構築したNNのインスタンスを定義して、実体を作る
model = NeuralNetwork()
print(model)

NameError: name 'nn' is not defined

nn.Flatten()というモジュールが出てきました。これは入力のデータの次元を1次元テンソルに変換するモジュールです(引数によっては2次元もいける)。
なぜ、画像を一次元にするかというと、下で定義したnn.Sequentialの入力が一次元であるためです。
このnn.SequentialはNNの本体のことで、入力が28*28の一次元となっているため入力データを一次元にする必要があります。

5、損失関数とオプティマイザ：
続いて損失関数とオプティマイザを用意します。
損失関数はNNの出力結果と実際の答えとの差を求めるものでしたね。pytorchでは標準搭載されている損失関数がいくつかあり、ここでは「CrossEntoropyLoss()」関数を用います

In [7]:
loss_fn = nn.CrossEntropyLoss()
#nnクラスのモジュールです

オプティマイザは損失関数で求めた誤差を小さくするためにNNのパラメータ(重みとバイアス)
を調整するものでしたね。こちらもpytorchで標準搭載されているものがあり「SDG」というものを使います

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
#torchの中のoptimクラスのSGDモジュールです

このSGDには主に2つの引数があります。NNのパラメータ(重みとバイアス)と学習率です
学習率とは、パラメータの調節の大きさを示すものであり、大きくすると学習が早くなる分精度が出なくなり、小さくするとその逆になります。

6、学習と検証：
いよいよ学習の準備ですここでは自前の学習用関数を用意します

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    #データセットのサイズを読み込み
    size = len(dataloader.dataset)
    #NNを訓練モードに変更
    model.train()
    #データローダーから画像とラベルを読み込む
    for batch, (X, y) in enumerate(dataloader):
        #NNの予測した値
        pred = model(X)
        #誤差(損失)を取得
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        #逆誤差伝搬
        loss.backward()
        #オプティマイザがパラメータを調整
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

つぎに、NNの認識精度を示す検証関数を作ります

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    #NNを検証モードに変更
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            #誤差(損失)の合計を計算
            test_loss += loss_fn(pred, y).item()
            #NNの予測した判定のうち、正解した回数の合計
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

学習開始です!

In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.161135  [    0/60000]
loss: 2.144341  [ 6400/60000]
loss: 2.098985  [12800/60000]
loss: 2.119114  [19200/60000]
loss: 2.030149  [25600/60000]
loss: 1.990340  [32000/60000]
loss: 2.011037  [38400/60000]
loss: 1.934530  [44800/60000]
loss: 1.954581  [51200/60000]
loss: 1.868467  [57600/60000]
Test Error: 
 Accuracy: 48.2%, Avg loss: 1.869736 

Epoch 2
-------------------------------
loss: 1.902672  [    0/60000]
loss: 1.861884  [ 6400/60000]
loss: 1.760754  [12800/60000]
loss: 1.808159  [19200/60000]
loss: 1.663081  [25600/60000]
loss: 1.636727  [32000/60000]
loss: 1.657014  [38400/60000]
loss: 1.568646  [44800/60000]
loss: 1.601552  [51200/60000]
loss: 1.496796  [57600/60000]
Test Error: 
 Accuracy: 59.7%, Avg loss: 1.512913 

Epoch 3
-------------------------------
loss: 1.574377  [    0/60000]
loss: 1.537365  [ 6400/60000]
loss: 1.402971  [12800/60000]
loss: 1.476188  [19200/60000]
loss: 1.341795  [25600/60000]
loss: 1.348737  [32000/600

7、NNの保存と使用：
では、学習させたNNを保存してみましょう。pytorchのNNは、「pth」ファイルで保存されます。

In [13]:
torch.save(model, "model.pth")
print("Saved PyTorch Model to model.pth")

Saved PyTorch Model to model.pth


保存したNNは、以下のようなコードで読み込めます

In [14]:
model=torch.load("model.pth")

では、読み込んだNNを利用してみましょう

In [ ]:
#ラベル番号と画像に写るものの名前を対応させた配列
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
#検証モード
model.eval()
#検証データセットの１枚目の画像とラベルを読み込み
x, y = test_data[0][0], test_data[0][1]

#この「with torch.no_grad」がないと、pythonが落ちる
with torch.no_grad():
    #NNの予測結果を代入
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

どうでしょうか?NNの回答と正解が一致していましたか?
以上で第一回は終了となります。

コード引用:https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html